<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-google-colab/blob/main/jupyter-notebooks/text-to-vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [127]:
# load static files

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [128]:
# imports

import os
import glob
# from dotenv import load_dotenv
import gradio as gr
from google.colab import userdata
from openai import OpenAI

In [129]:
!pip install langchain-core langchain-text-splitters langchain-openai langchain-chroma langchain-community

In [130]:
# imports for langchain and Chroma and plotly

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [131]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [132]:
# keys

openai_api_key = userdata.get("OPENAI_API_KEY")
# claude_api_key = userdata.get("ANTHROPIC_API_KEY")
# google_api_key = userdata.get("GOOGLE_API_KEY")
# hugging_face_token = userdata.get("HF_TOKEN")

# load_dotenv(override=True)
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [133]:
# initialize

# openai = OpenAI(api_key=openai_api_key)
# api_key = openai_api_key
os.environ['OPENAI_API_KEY']  = openai_api_key

In [134]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase
# Thank you Mark D. and Zoya H. for fixing a bug here..

folders = glob.glob("/content/drive/MyDrive/Colab Notebooks/knowledge-base/*")
folders = glob.glob("/content/drive/MyDrive/Colab Notebooks/knowledge-base-languages/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [135]:
len(documents)

26

In [136]:
documents[24]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/knowledge-base-languages/languages/gascon.md', 'doc_type': 'languages'}, page_content="# Overview of the Gascon Language\n\n## General Information\n- **Language Family**: Occitan branch of the Romance languages.\n- **Region**: Primarily spoken in the Gascony region of southwestern France, which includes parts of the departments of Gers, Landes, and Pyrénées-Atlantiques.\n  \n## Historical Context\n- **Origins**: Gascon evolved from Vulgar Latin and has influences from the Visigoths and various other historical invaders.\n- **Status**: Once a widely spoken language, Gascon has seen a decline in the number of speakers, particularly in urban areas, due to the rise of French as the dominant language.\n\n## Dialects\n- **Varieties**: Gascon includes several dialects, most notably:\n  - **Bigourdan**: Spoken in the region of Bigorre.\n  - **Armanac**: Found in Armagnac.\n  - **Languedocien**: This influences some Gascon spe

In [137]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [138]:
len(chunks)

116

In [139]:
chunks[6]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/knowledge-base-languages/regions/gascogne.md', 'doc_type': 'regions'}, page_content='## Conclusion\nGascogne is a region that offers a unique blend of history, culture, and natural beauty. From its medieval villages and legendary connections to the Musketeers, to its rich culinary traditions and scenic landscapes, Gascogne provides a true taste of southwestern France. Whether exploring its vineyards, tasting Armagnac, or immersing yourself in its rural charm, Gascogne is a region full of life and tradition.')

In [140]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: regions, mountains, languages


In [141]:
for chunk in chunks:
    if 'Lancaster' in chunk.page_content:
        print(chunk)
        print("_________")

## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [142]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [143]:
# Check if a Chroma Datastore already exists - if so, delete the collection to start from scratch

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [144]:
# Create our Chroma vectorstore!

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 116 documents


In [145]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [147]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
# colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]
colors = [['blue', 'green', 'red'][['languages', 'regions', 'mountains'].index(t)] for t in doc_types]

In [148]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [149]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()